<a href="https://colab.research.google.com/github/dframbaut/web-scrapping-deepear/blob/main/web_scrapping_simi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 18.2 MB/s eta 0:00:00


In [2]:
!apt-get update

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,192 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,454 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois

In [3]:
!apt-get install -y chromium-chromedriver

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  apparmor chromium-browser libfuse3-3 liblzo2-2 libudev1 snapd squashfs-tools systemd-hwe-hwdb
  udev
Suggested packages:
  apparmor-profiles-extra apparmor-utils fuse3 zenity | kdialog
The following NEW packages will be installed:
  apparmor chromium-browser chromium-chromedriver libfuse3-3 liblzo2-2 snapd squashfs-tools
  systemd-hwe-hwdb udev
The following packages will be upgraded:
  libudev1
1 upgraded, 9 newly installed, 0 to remove and 49 not upgraded.
Need to get 30.2 MB of archives.
After this operation, 123 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 apparmor amd64 3.0.4-2ubuntu2.4 [598 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 liblzo2-2 amd64 2.10-2build3 [53.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 squashfs-tools amd64 1:4.5-3

In [4]:
!cp /usr/lib/chromium-browser/chromedriver /usr/bin


cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file


In [7]:
# Configurar Selenium para Colab
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

# Configuración del navegador en modo headless
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Modo sin interfaz gráfica
options.add_argument('--no-sandbox')  # Necesario para Colab
options.add_argument('--disable-dev-shm-usage')  # Evitar problemas de memoria
options.add_argument('--disable-gpu')

# Inicializar el controlador del navegador
driver = webdriver.Chrome(options=options)

# URL base
url = "https://simi.concejodemedellin.gov.co/simi3/invitados/proyectos/acuerdos.xhtml"

# Inicializar listas para almacenar los datos
Nombres = []
Fechas = []
Epigrafe = []
Tipo = []
botones = []

try:
    # Navegar al sitio web
    driver.get(url)

    # Esperar a que la página cargue completamente
    time.sleep(5)

    while True:  # Bucle para recorrer todas las páginas
        print("Procesando página...")

        # Localizar la tabla en la página actual
        table = driver.find_element(By.TAG_NAME, "table")
        tbody = table.find_element(By.TAG_NAME, "tbody")
        rows = tbody.find_elements(By.TAG_NAME, "tr")

        # Procesar cada fila de la tabla
        for row_index, row in enumerate(rows, start=1):
            cells = row.find_elements(By.TAG_NAME, "td")
            for col_index, cell in enumerate(cells, start=1):
                cell_content = cell.text.strip()
                if col_index == 1:  # Nombre
                    Nombres.append(cell_content)
                elif col_index == 2:  # Fecha
                    Fechas.append(cell_content)
                elif col_index == 3:  # Epígrafe
                    Epigrafe.append(cell_content)
                elif col_index == 4:  # Tipo
                    Tipo.append(cell_content)
                elif col_index == 5:  # Enlace
                    try:
                        botones.append(cell.find_element(By.TAG_NAME, "a").get_attribute("href"))
                    except:
                        botones.append(None)  # Si no hay enlace

        # Intentar hacer clic en el siguiente botón numérico
        try:
            # Buscar todos los botones de paginación (números de página)
            page_buttons = driver.find_elements(By.CSS_SELECTOR, ".ui-paginator-page")#Busca todos los marcados por ".ui-paginator-page"

            # Buscar el siguiente botón de la paginación
            for button in page_buttons:
                if button.text and button.text.isdigit():
                    page_number = int(button.text)
                    if page_number > 1:  # Ignorar la página 1, ya que es la página inicial
                        button.click()
                        time.sleep(3)  # Esperar a que la página cargue
                        break
            if len(page_buttons) == 0 or page_buttons[-1].text == '10':  # Recorrer de la pagina 0 hasta la 10
                print("Última página alcanzada.")
                break

        except Exception as e:
            print("Error al navegar a la siguiente página:", e)
            break

    # Imprimir las listas de datos extraídos
    print("Nombres:", Nombres)
    print("Fechas:", Fechas)
    print("Epígrafe:", Epigrafe)
    print("Tipo:", Tipo)
    print("Botones:", botones)

finally:
    # Cerrar el navegador
    driver.quit()


Procesando página...
Error al navegar a la siguiente página: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=131.0.6778.87); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
#0 0x5c22569fb34a <unknown>
#1 0x5c22565116e0 <unknown>
#2 0x5c22565174a3 <unknown>
#3 0x5c2256519968 <unknown>
#4 0x5c22565199f3 <unknown>
#5 0x5c225655aba2 <unknown>
#6 0x5c2256584462 <unknown>
#7 0x5c2256554a18 <unknown>
#8 0x5c225658462e <unknown>
#9 0x5c22565a2ed7 <unknown>
#10 0x5c2256584203 <unknown>
#11 0x5c2256552cc0 <unknown>
#12 0x5c2256553c9e <unknown>
#13 0x5c22569c8d3b <unknown>
#14 0x5c22569cccc2 <unknown>
#15 0x5c22569b5b6c <unknown>
#16 0x5c22569cd837 <unknown>
#17 0x5c225699b10f <unknown>
#18 0x5c22569ea5a8 <unknown>
#19 0x5c22569ea770 <unknown>
#20 0x5c22569fa1c6 <unknown>
#21 0x7f0d4541dac3 <unknown>

Nombres: ['16-2024', '14-2